In [ ]:
#i/o 최적화, numpy 최적화를 적용해서 코드를 실행했지만 유의미한 실행속도의 차이를 찾지 못해 2번째, 3번째 코드로 이미지 전처리와 특징 추출 과정을 수행하였습니다.
#실행 결과의 경우, 너무 출력이 길어서 오류가 뜨는 바람에 코드만 첨부합니다.

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from tqdm.notebook import tqdm
from glob import glob
import time

# 이미지 로드
def load_images(path):
    image_files = sorted(glob(os.path.join(path, '*.BMP')))
    return image_files

# 이미지 이진화
def binarize_image(img):
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    return binary

# 이미지 세선화
def thin_image(img):
    skeleton = skeletonize(img // 255)
    return (skeleton * 255).astype(np.uint8)

# minutiae(end, bif) 탐지 (벡터화된 연산 활용)
def detect_minutiae(skeleton):
    minutiae_end = []
    minutiae_bif = []

    # 패딩 추가
    padded_skeleton = np.pad(skeleton, ((1, 1), (1, 1)), mode='constant')

    for i in range(1, padded_skeleton.shape[0] - 1):
        for j in range(1, padded_skeleton.shape[1] - 1):
            if padded_skeleton[i, j] == 255:
                block = padded_skeleton[i-1:i+2, j-1:j+2]
                white_pixels = np.sum(block == 255)
                if white_pixels == 2:
                    minutiae_end.append((i-1, j-1))
                elif white_pixels == 4:
                    minutiae_bif.append((i-1, j-1))
    
    return minutiae_end, minutiae_bif

# 이미지 사진 -> 원본, 이진화된 이미지, 그 후 세선화된 이미지, 마지막으로 minutiae(end, bif)를 표시한 이미지를 순서대로 나타냄
def plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif):
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 4, 2)
    plt.imshow(binary_img, cmap='gray')
    plt.title('Binarized Image')

    plt.subplot(1, 4, 3)
    plt.imshow(skeleton_img, cmap='gray')
    plt.title('Thinned Image')

    plt.subplot(1, 4, 4)
    plt.imshow(skeleton_img, cmap='gray')
    for (i, j) in minutiae_end:
        plt.plot(j, i, 'ro')  # red -> end
    for (i, j) in minutiae_bif:
        plt.plot(j, i, 'bo')  # blue -> bif
    plt.title('Minutiae Extraction Results')

    plt.show()

# 이미지 로드 후 함수 실행하는 메인 함수
def process_fingerprints(directory):
    image_files = load_images(directory)

    processing_times = []

    for file in tqdm(image_files):
        start_time = time.time()

        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        original_img = cv2.cvtColor(cv2.imread(file), cv2.COLOR_BGR2RGB)

        binary_img = binarize_image(img)
        skeleton_img = thin_image(binary_img)
        minutiae_end, minutiae_bif = detect_minutiae(skeleton_img)

        processing_time = time.time() - start_time
        processing_times.append(processing_time)

        print(f"Processed {file} in {processing_time:.2f} seconds")
        plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif)

    avg_time = np.mean(processing_times)
    print(f"Average processing time: {avg_time:.2f} seconds")

train_path = './train_ref'
process_fingerprints(train_path)


In [ ]:
#이미지 전처리 - 특징 추출

In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from tqdm.notebook import tqdm
from glob import glob
import time

#test2 폴더와 train_ref 폴더를 미리 업로드 후 코드 실행

# 이미지 로드
def load_images(path):
    image_files = sorted(glob(os.path.join(path, '*.BMP')))
    return image_files

# 이미지 이진화
def binarize_image(img):
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    return binary

# 이미지 세선화
def thin_image(img):
    skeleton = skeletonize(img // 255)
    return (skeleton * 255).astype(np.uint8)

# minutiae(end, bif) 탐지
def detect_minutiae(skeleton):
    minutiae_end = []
    minutiae_bif = []
    for i in range(1, skeleton.shape[0] - 1):
        for j in range(1, skeleton.shape[1] - 1):
            if skeleton[i, j] == 255:
                block = skeleton[i-1:i+2, j-1:j+2]
                if np.sum(block) == 255 * 2:
                    minutiae_end.append((i, j))
                elif np.sum(block) == 255 * 4:
                    minutiae_bif.append((i, j))
    return minutiae_end, minutiae_bif

# 이미지 사진 -> 원본, 이진화된 이미지, 그 후 세선화된 이미지, 마지막으로 minutiae(end, bif)를 표시한 이미지를 순서대로 나타냄
def plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif):
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 4, 2)
    plt.imshow(binary_img, cmap='gray')
    plt.title('Binarized Image')

    plt.subplot(1, 4, 3)
    plt.imshow(skeleton_img, cmap='gray')
    plt.title('Thinned Image')

    plt.subplot(1, 4, 4)
    plt.imshow(skeleton_img, cmap='gray')
    for (i, j) in minutiae_end:
        plt.plot(j, i, 'ro')  # red -> end
    for (i, j) in minutiae_bif:
        plt.plot(j, i, 'bo')  # blue -> bif
    plt.title('Minutiae Extraction Results')

    plt.show()

# 이미지 로드 후 함수 실행하는 메인 함수
def process_fingerprints(directory):
    image_files = load_images(directory)

    processing_times = []

    for file in tqdm(image_files):
        start_time = time.time()

        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        original_img = cv2.imread(file)

        binary_img = binarize_image(img)
        skeleton_img = thin_image(binary_img)
        minutiae_end, minutiae_bif = detect_minutiae(skeleton_img)

        processing_time = time.time() - start_time
        processing_times.append(processing_time)

        print(f"Processed {file} in {processing_time:.2f} seconds")
        plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif)

    avg_time = np.mean(processing_times)
    print(f"Average processing time: {avg_time:.2f} seconds")

train_path = './train_ref'
process_fingerprints(train_path)


In [ ]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
from skimage.morphology import skeletonize
from tqdm.notebook import tqdm
from glob import glob
import time

#test2 폴더와 train_ref 폴더를 미리 업로드 후 코드 실행

# 이미지 로드
def load_images(path):
    image_files = sorted(glob(os.path.join(path, '*.BMP')))
    return image_files

# 이미지 이진화
def binarize_image(img):
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    return binary

# 이미지 세선화
def thin_image(img):
    skeleton = skeletonize(img // 255)
    return (skeleton * 255).astype(np.uint8)

# minutiae(end, bif) 탐지
def detect_minutiae(skeleton):
    minutiae_end = []
    minutiae_bif = []
    for i in range(1, skeleton.shape[0] - 1):
        for j in range(1, skeleton.shape[1] - 1):
            if skeleton[i, j] == 255:
                block = skeleton[i-1:i+2, j-1:j+2]
                if np.sum(block) == 255 * 2:
                    minutiae_end.append((i, j))
                elif np.sum(block) == 255 * 4:
                    minutiae_bif.append((i, j))
    return minutiae_end, minutiae_bif

# 이미지 사진 -> 원본, 이진화된 이미지, 그 후 세선화된 이미지, 마지막으로 minutiae(end, bif)를 표시한 이미지를 순서대로 나타냄
def plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif):
    plt.figure(figsize=(20, 5))

    plt.subplot(1, 4, 1)
    plt.imshow(original_img, cmap='gray')
    plt.title('Original Image')

    plt.subplot(1, 4, 2)
    plt.imshow(binary_img, cmap='gray')
    plt.title('Binarized Image')

    plt.subplot(1, 4, 3)
    plt.imshow(skeleton_img, cmap='gray')
    plt.title('Thinned Image')

    plt.subplot(1, 4, 4)
    plt.imshow(skeleton_img, cmap='gray')
    for (i, j) in minutiae_end:
        plt.plot(j, i, 'ro')  # red -> end
    for (i, j) in minutiae_bif:
        plt.plot(j, i, 'bo')  # blue -> bif
    plt.title('Minutiae Extraction Results')

    plt.show()

# 이미지 로드 후 함수 실행하는 메인 함수
def process_fingerprints(directory):
    image_files = load_images(directory)

    processing_times = []

    for file in tqdm(image_files):
        start_time = time.time()

        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        original_img = cv2.imread(file)

        binary_img = binarize_image(img)
        skeleton_img = thin_image(binary_img)
        minutiae_end, minutiae_bif = detect_minutiae(skeleton_img)

        processing_time = time.time() - start_time
        processing_times.append(processing_time)

        print(f"Processed {file} in {processing_time:.2f} seconds")
        plot_images(original_img, binary_img, skeleton_img, minutiae_end, minutiae_bif)

    avg_time = np.mean(processing_times)
    print(f"Average processing time: {avg_time:.2f} seconds")

train_path = './test2'
process_fingerprints(train_path)


In [ ]:
#특징 추출 후 파일 저장

In [ ]:
import os
import cv2
import numpy as np
from skimage.morphology import skeletonize
from glob import glob
import pickle
from tqdm.notebook import tqdm

# pkl 파일 추출
# 각각 train_ref, test2에 속한 파일들에 대한 pkl 파일(minutiae 특징 추출해서 모아놓은 파일) 저장

def load_images(path):
    image_files = sorted(glob(os.path.join(path, '*.BMP')))
    return image_files

def binarize_image(img):
    _, binary = cv2.threshold(img, 128, 255, cv2.THRESH_BINARY_INV)
    return binary

def thin_image(img):
    skeleton = skeletonize(img // 255)
    return (skeleton * 255).astype(np.uint8)

def detect_minutiae(skeleton):
    minutiae_end = []
    minutiae_bif = []
    for i in range(1, skeleton.shape[0] - 1):
        for j in range(1, skeleton.shape[1] - 1):
            if skeleton[i, j] == 255:
                block = skeleton[i-1:i+2, j-1:j+2]
                if np.sum(block) == 255 * 2:
                    minutiae_end.append((i, j))
                elif np.sum(block) == 255 * 4:
                    minutiae_bif.append((i, j))
    return minutiae_end, minutiae_bif

def extract_and_save_features(directory, output_file):
    image_files = load_images(directory)
    features = {}

    for file in tqdm(image_files, desc=f"Processing {directory}"):
        img = cv2.imread(file, cv2.IMREAD_GRAYSCALE)
        binary_img = binarize_image(img)
        skeleton_img = thin_image(binary_img)
        minutiae_end, minutiae_bif = detect_minutiae(skeleton_img)
        features[os.path.basename(file)] = (minutiae_end, minutiae_bif)

    with open(output_file, 'wb') as f:
        pickle.dump(features, f)

train_path = './train_ref'
test_path = './test2'

extract_and_save_features(train_path, 'train_features.pkl')
extract_and_save_features(test_path, 'test_features.pkl')


In [ ]:
#이미지 매칭 및 threshold를 1.0으로 설정 후 Metric 계산

In [ ]:
import os
import numpy as np
from scipy.spatial.distance import cdist
import pickle
from tqdm.notebook import tqdm


# 2개의 minutiae 세트를 통해 Euclidean 거리 계산
def calculate_distance(minutiae1, minutiae2):
    if len(minutiae1) == 0 or len(minutiae2) == 0:
        return float('inf')
    minutiae1 = np.array(minutiae1)
    minutiae2 = np.array(minutiae2)
    dists = cdist(minutiae1, minutiae2, 'euclidean')
    min_dists = np.min(dists, axis=1)
    return np.mean(min_dists)


# pkl파일을 불러옴
def load_features(file):
    with open(file, 'rb') as f:
        features = pickle.load(f)
    return features


# 매칭된 지문 사진 가져옴 / 구해진 거리(최소 거리)를 통해 threshold 계산
def match_fingerprints_and_evaluate(train_features_file, test_features_file, threshold):
    train_features = load_features(train_features_file)
    test_features = load_features(test_features_file)
   
    results = {}
   
    for test_image, (test_minutiae_end, test_minutiae_bif) in tqdm(test_features.items(), desc="Matching test images"):
        test_minutiae = test_minutiae_end + test_minutiae_bif
        closest_image = None
        min_distance = float('inf')
       
        for train_image, (train_minutiae_end, train_minutiae_bif) in train_features.items():
            train_minutiae = train_minutiae_end + train_minutiae_bif
            dist = calculate_distance(test_minutiae, train_minutiae)
            if dist < min_distance:
                min_distance = dist
                closest_image = train_image
               
        results[test_image] = (closest_image, min_distance)
        print(f"Test Image: {test_image}, Closest Train Image: {closest_image}, Distance: {min_distance}")
   
    # Metric 평가
    tp = fp = tn = fn = 0
    for test_image, (train_image, distance) in results.items():
        test_id = test_image.split('_')[0]
        train_id = train_image.split('.')[0]
        if distance < threshold:
            if test_id == train_id:
                tp += 1  # TP
            else:
                fp += 1  # FP
        else:
            if test_id == train_id:
                fn += 1  # FN
            else:
                tn += 1  # TN


    precision = tp / (tp + fp) if (tp + fp) > 0 else 0
    recall = tp / (tp + fn) if (tp + fn) > 0 else 0
    far = fp / (fp + tn) if (fp + tn) > 0 else 0  # False Acceptance Rate
    frr = fn / (fn + tp) if (fn + tp) > 0 else 0  # False Rejection Rate
    accuracy = (tp + tn) / (tp + tn + fp + fn) if (tp + tn + fp + fn) > 0 else 0


    print(f"\nMetrics with threshold {threshold}:")
    print(f"Precision: {precision:.2f}")
    print(f"Recall: {recall:.2f}")
    print(f"False Acceptance Rate (FAR): {far:.2f}")
    print(f"False Rejection Rate (FRR): {frr:.2f}")
    print(f"Accuracy: {accuracy:.2f}")


# 이전에 저장한 pkl 파일
train_features_file = 'train_features.pkl'
test_features_file = 'test_features.pkl'


# 적절한 threshold 설정
threshold = 1.0


match_fingerprints_and_evaluate(train_features_file, test_features_file, threshold)
